In [1]:

# Data Cleaning
import pandas as pd
import re
data = pd.read_csv('100_movies_imdb.csv')
data['Commentaire'] = data['Commentaire'].apply(lambda x: re.sub('<.*?>', '', x))
data['Commentaire'] = data['Commentaire'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]', '', x))
data['Commentaire'] = data['Commentaire'].str.lower()
data['Commentaire'] = data['Commentaire'].apply(lambda x: ' '.join(x.split()))
print(data.head())
data.describe()
data.isna().sum()


<>:12: SyntaxWarning: invalid escape sequence '\s'
<>:12: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Salma Fannich\AppData\Local\Temp\ipykernel_13212\2564944264.py:12: SyntaxWarning: invalid escape sequence '\s'
  data['Commentaire'] = data['Commentaire'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]', '', x))


              Film                                        Commentaire
0  Project Almanac  the teenager david raskin jonny weston is a ge...
1  Project Almanac  project almanac is a sort of the butterfly eff...
2  Project Almanac     title brazil projeto almanaque project almanac
3  Project Almanac  and this is true of both the characters in the...
4  Project Almanac  but throw logic out of the window because this...


Film           0
Commentaire    0
dtype: int64

### Informations about our data 


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2543 entries, 0 to 2542
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Film         2543 non-null   object
 1   Commentaire  2543 non-null   object
dtypes: object(2)
memory usage: 39.9+ KB


### deleting the duplicated data 

In [3]:
data = data.drop_duplicates()
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 2541 entries, 0 to 2542
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Film         2541 non-null   object
 1   Commentaire  2541 non-null   object
dtypes: object(2)
memory usage: 59.6+ KB
None


### Adding a colone for sentiments of reviews based on our model SVM 

In [4]:
import pickle
import pandas as pd

with open('sentiment_model.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

with open('tfidfvectorizer.pkl', 'rb') as vectorizer_file:
    loaded_vectorizer = pickle.load(vectorizer_file)

data = pd.read_csv('100_movies_imdb.csv')

data['Predicted_Sentiment'] = None

comments_tfidf = loaded_vectorizer.transform(data['Commentaire'])

predicted_sentiments = loaded_model.predict(comments_tfidf)

data['Predicted_Sentiment'] = predicted_sentiments
sentiment_percentage = data['Predicted_Sentiment'].value_counts(normalize=True) * 100
print("Pourcentage des sentiments pour chaque classe :")
print(sentiment_percentage)


Pourcentage des sentiments pour chaque classe :
Predicted_Sentiment
positif    77.113645
négatif    15.768777
neutre      7.117578
Name: proportion, dtype: float64


In [9]:
import pickle
import pandas as pd

with open('sentiment_model.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

with open('tfidfvectorizer.pkl', 'rb') as vectorizer_file:
    loaded_vectorizer = pickle.load(vectorizer_file)

data = pd.read_csv('100_movies_imdb.csv')

comments_tfidf = loaded_vectorizer.transform(data['Commentaire'])

predicted_sentiments = loaded_model.predict(comments_tfidf)

result_data = pd.DataFrame({
    'Film': data['Film'],
    'Commentaire': data['Commentaire'],
    'Predicted_Sentiment': predicted_sentiments
})

print(result_data)
result_data.to_csv('100_movie_sentiment_svm.csv', index=False)


                 Film                                        Commentaire  \
0     Project Almanac  The teenager David Raskin (Jonny Weston) is a ...   
1     Project Almanac  "Project Almanac" is a sort of "The Butterfly ...   
2     Project Almanac  Title (Brazil): "Projeto Almanaque" ("Project ...   
3     Project Almanac  And this is true of both the characters in the...   
4     Project Almanac  But throw logic out of the window, because thi...   
...               ...                                                ...   
2538  Pitch Perfect 2  If you enjoyed the 2012 original, I will surel...   
2539  Pitch Perfect 2  The film has new characters which are quite fu...   
2540  Pitch Perfect 2  The film does have some plot holes that I noti...   
2541  Pitch Perfect 2  The good points, well where do I start! Fat Am...   
2542  Pitch Perfect 2  In a surprise hit, pitch perfect managed to sh...   

     Predicted_Sentiment  
0                négatif  
1                négatif  
2     

### Create new dataframe with movie name and sentiment prediction using our model 

In [10]:
import pickle
import pandas as pd

with open('sentiment_model.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

with open('tfidfvectorizer.pkl', 'rb') as vectorizer_file:
    loaded_vectorizer = pickle.load(vectorizer_file)

data2 = pd.read_csv('imdb_movies_sentiments_svm.csv')
comments_tfidf = loaded_vectorizer.transform(data2['Commentaire'])

predicted_sentiments = loaded_model.predict(comments_tfidf)

data2['Predicted_Sentiment'] = predicted_sentiments

data2.to_csv('imdb_movies_sentiments_svm_with_predictions.csv', index=False)

print(data2[['Commentaire', 'Predicted_Sentiment']])


                                             Commentaire Predicted_Sentiment
0      You'll have to have your wits about you and yo...             positif
1      One of the most anticipated films of the year ...             positif
2      But in many ways, I can't deny it was an excep...             positif
3      The film's at its best when it focuses on bein...             positif
4      I did really feel the length in the final hour...             positif
...                                                  ...                 ...
22271  This movie is so underrated. The suspense is r...             positif
22272  I don't think a lot of the negative reviewers ...             négatif
22273  What does that even have to do with the movie?...             positif
22274  How would anybody alive now even know exactly ...             positif
22275  I hate to break it to you, but if you actually...             négatif

[22276 rows x 2 columns]


In [13]:
import pandas as pd
data2 = pd.read_csv('imdb_movies_sentiments_svm_with_predictions.csv')
data2.drop('Commentaire', axis=1, inplace=True)

In [14]:
import pandas as pd

data2 = pd.read_csv('imdb_movies_sentiments_svm_with_predictions.csv')
data2.drop('Commentaire', axis=1, inplace=True)
print(data2)


              Film Predicted_Sentiment
0      Oppenheimer             positif
1      Oppenheimer             positif
2      Oppenheimer             positif
3      Oppenheimer             positif
4      Oppenheimer             positif
...            ...                 ...
22271   Apocalypto             positif
22272   Apocalypto             négatif
22273   Apocalypto             positif
22274   Apocalypto             positif
22275   Apocalypto             négatif

[22276 rows x 2 columns]


### poucentage for each movie sentiment

In [17]:
import pandas as pd


data2 = pd.read_csv('imdb_movies_sentiments_svm_with_predictions.csv')
sentiment_percentages = data2.groupby(['Film', 'Predicted_Sentiment']).size().unstack(fill_value=0)
sentiment_percentages['Total'] = sentiment_percentages.sum(axis=1)

#  le pourcentage de chaque sentiment par film
sentiment_percentages['Positive_Percentage'] = sentiment_percentages['positif'] / sentiment_percentages['Total'] * 100
sentiment_percentages['Negative_Percentage'] = sentiment_percentages['négatif'] / sentiment_percentages['Total'] * 100
sentiment_percentages['Neutral_Percentage'] = sentiment_percentages['neutre'] / sentiment_percentages['Total'] * 100
formatted_percentages = sentiment_percentages[['Positive_Percentage', 'Negative_Percentage', 'Neutral_Percentage']].round(2)
print(formatted_percentages)


Predicted_Sentiment                   Positive_Percentage  \
Film                                                        
 2001: A Space Odyssey                              77.19   
 Ah-ga-ssi                                          79.59   
 Dunkirk                                            68.75   
 Ghostbusters                                       82.76   
 Harry Potter and the Goblet of Fire                66.42   
...                                                   ...   
Watchmen                                            84.55   
Whiplash                                            72.13   
Wind River                                          73.03   
Zack Snyder's Justice League                        73.85   
Zodiac                                              78.85   

Predicted_Sentiment                   Negative_Percentage  Neutral_Percentage  
Film                                                                           
 2001: A Space Odyssey                        

In [22]:
import pandas as pd

data2 = pd.read_csv('imdb_movies_sentiments_svm_with_predictions.csv')

film_to_check = "The Lord of the Rings: The Fellowship of the Ring"

film_data = data2[data2['Film'] == film_to_check]

sentiment_percentages = film_data.groupby(['Predicted_Sentiment']).size() / len(film_data) * 100

print(f"Pourcentage de sentiments pour le film '{film_to_check}':")
print(sentiment_percentages)


Pourcentage de sentiments pour le film 'The Lord of the Rings: The Fellowship of the Ring':
Predicted_Sentiment
neutre      7.207207
négatif     3.603604
positif    89.189189
dtype: float64
